# Telco Customer Churn

link: https://www.kaggle.com/datasets/blastchar/telco-customer-churn

## Bibliotecas 

In [1]:
# manipulação de dados
import pandas as pd
import numpy as np

# pre-processamento
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

# separação treino e teste
from sklearn.model_selection import train_test_split

# modelos
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
from tensorflow import keras
from tensorflow.keras import layers

# metricas
from sklearn.metrics import accuracy_score, confusion_matrix

## Leitura dos dados 

In [2]:
df_raw = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')
df_raw.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


## Verificação dos dados

### tamanho dos dados

In [3]:
print('Total de linhas: ', df_raw.shape[0])
print('Total de colunas: ', df_raw.shape[1])

Total de linhas:  7043
Total de colunas:  21


### vertificar dados faltantes

In [4]:
df_raw.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

### verificar dados duplicados

In [5]:
df_raw.duplicated().sum()

0

### tipos dos dados

In [18]:
df_raw.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

## Seleção de features

**Não precisando do customerID**

In [7]:
df = df_raw.copy()
# não precisamos do customerID
df = df.drop('customerID', axis = 1)
#df = df.dropna(subset=['TotalCharges'])

# a variavel TotalCharges está como categorico
df['TotalCharges'] = df['TotalCharges'].replace(' ', 0).astype(float)


In [19]:
df.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges        float64
Churn                object
dtype: object

In [8]:
# target
#y = df['Churn']
y = pd.get_dummies(df['Churn'], drop_first = True)
y['Target'] = y['Yes']
y.drop('Yes', axis = 1, inplace=True)

# x categorico
onehotencoder = OneHotEncoder()
variaveis_categoricas = df.select_dtypes(include=['object']).drop('Churn', axis=1)
x_categorico = onehotencoder.fit_transform(variaveis_categoricas).toarray()
x_categorico = pd.DataFrame(x_categorico)

# x numerico
variaveis_numericas = df.select_dtypes(include=['int','float'])
x_numerico = variaveis_numericas

# variavel categorica
x_all = pd.concat([x_categorico, x_numerico], axis=1)
scaler = MinMaxScaler()
x = scaler.fit_transform(x_all)
x


C:\Users\Ben-Hur\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\Ben-Hur\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


array([[1.        , 0.        , 0.        , ..., 0.01388889, 0.11542289,
        0.00343704],
       [0.        , 1.        , 1.        , ..., 0.47222222, 0.38507463,
        0.21756402],
       [0.        , 1.        , 1.        , ..., 0.02777778, 0.35422886,
        0.01245279],
       ...,
       [1.        , 0.        , 0.        , ..., 0.15277778, 0.11293532,
        0.03989153],
       [0.        , 1.        , 0.        , ..., 0.05555556, 0.55870647,
        0.03530306],
       [0.        , 1.        , 1.        , ..., 0.91666667, 0.86965174,
        0.78810105]])

In [11]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.25)

In [10]:
x_train.shape

(5282, 45)

In [11]:
x_test.shape

(1761, 45)

In [12]:
y_train.shape

(5282, 1)

In [13]:
y_test.shape

(1761, 1)

## Regressão logistica

In [12]:
logistic_model = LogisticRegression()
logistic_model.fit(x_train, y_train)

C:\Users\Ben-Hur\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Ben-Hur\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [13]:
y_pred = logistic_model.predict(x_test)
y_pred

array([1, 0, 0, ..., 0, 0, 0], dtype=uint8)

In [14]:
accuracy_score(y_test, y_pred)

0.814877910278251

In [15]:
cm = confusion_matrix(y_test, y_pred)
cm

array([[1166,  118],
       [ 208,  269]], dtype=int64)

In [17]:
from joblib import dump

# Suponha que 'modelo' seja o seu modelo treinado
dump(logistic_model, 'logistic_model.joblib')


['logistic_model.joblib']

## Arvore de decisão

In [43]:
%%time
tree_model = DecisionTreeClassifier(max_depth=2,
                                   min_samples_leaf = 5)
tree_model.fit(x_train, y_train)

Wall time: 8.01 ms


DecisionTreeClassifier(max_depth=2, min_samples_leaf=5)

In [44]:
y_pred = tree_model.predict(x_test)
y_pred

array([1, 1, 0, ..., 0, 0, 0], dtype=uint8)

In [23]:
accuracy1 = accuracy_score(y_test, y_pred)

0.7637705848949461

In [45]:
accuracy2 = accuracy_score(y_test, y_pred)

In [37]:
cm = confusion_matrix(y_test, y_pred)
cm

array([[1074,  235],
       [ 181,  271]], dtype=int64)

In [29]:
def treina_arvore_decisao(x_train, y_train, x_test, y_test, max_d, min_s):
    modelo = DecisionTreeClassifier(max_depth = max_d,
                                   min_samples_leaf = min_s)
    modelo.fit(x_train, y_train)
    
    y_pred = modelo.predict(x_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    return modelo, accuracy

In [38]:
%%time
modelo_list = list()
max_d = list()
min_s = list()
accuracy_list = list()

for d in range(1,10):
    for s in range(1 ,31):
        print('Treinando modelo', s)
        max_d.append(d)
        min_s.append(s)
        modelo, accuracy = treina_arvore_decisao(x_train, y_train, x_test, y_test, d, s)
        
        modelo_list.append(modelo)
        accuracy_list.append(accuracy)
        
busca_ad = pd.DataFrame(list(zip(max_d, min_s, accuracy_list, modelo_list)),
                       columns = ['Max Deph', 'Min Sample Leaf', 'Accuracy', 'Modelo'])
#busca_ad

Treinando modelo 1
Treinando modelo 2
Treinando modelo 3
Treinando modelo 4
Treinando modelo 5
Treinando modelo 6
Treinando modelo 7
Treinando modelo 8
Treinando modelo 9
Treinando modelo 10
Treinando modelo 11
Treinando modelo 12
Treinando modelo 13
Treinando modelo 14
Treinando modelo 15
Treinando modelo 16
Treinando modelo 17
Treinando modelo 18
Treinando modelo 19
Treinando modelo 20
Treinando modelo 21
Treinando modelo 22
Treinando modelo 23
Treinando modelo 24
Treinando modelo 25
Treinando modelo 26
Treinando modelo 27
Treinando modelo 28
Treinando modelo 29
Treinando modelo 30
Treinando modelo 1
Treinando modelo 2
Treinando modelo 3
Treinando modelo 4
Treinando modelo 5
Treinando modelo 6
Treinando modelo 7
Treinando modelo 8
Treinando modelo 9
Treinando modelo 10
Treinando modelo 11
Treinando modelo 12
Treinando modelo 13
Treinando modelo 14
Treinando modelo 15
Treinando modelo 16
Treinando modelo 17
Treinando modelo 18
Treinando modelo 19
Treinando modelo 20
Treinando modelo 2

In [39]:
busca_ad.sort_values('Accuracy', ascending=False)

,Max Deph,Min Sample Leaf,Accuracy,Modelo
109,4,20,0.797274,"DecisionTreeClassifier(max_depth=4, min_sample..."
119,4,30,0.797274,"DecisionTreeClassifier(max_depth=4, min_sample..."
118,4,29,0.797274,"DecisionTreeClassifier(max_depth=4, min_sample..."
117,4,28,0.797274,"DecisionTreeClassifier(max_depth=4, min_sample..."
116,4,27,0.797274,"DecisionTreeClassifier(max_depth=4, min_sample..."
...,...,...,...,...
15,1,16,0.743328,"DecisionTreeClassifier(max_depth=1, min_sample..."
14,1,15,0.743328,"DecisionTreeClassifier(max_depth=1, min_sample..."
13,1,14,0.743328,"DecisionTreeClassifier(max_depth=1, min_sample..."
12,1,13,0.743328,"DecisionTreeClassifier(max_depth=1, min_sample..."


In [41]:
modelo_selecionado = busca_ad.iloc[109]['Modelo']

## Redes Neurais

### Definição do NPL

In [18]:
modelo_rn = keras.Sequential()

# Camada Oculta 1
modelo_rn.add(layers.Dense(20,
                         activation='tanh',
                          input_dim=x_train.shape[1]))

# Camada Oculta 2
modelo_rn.add(layers.Dense(16,
                         activation = 'relu',
                         input_dim = x_train.shape[1]))

# Camada Saida
modelo_rn.add(layers.Dense(1,
                          activation = 'sigmoid'))


#
modelo_rn.compile(loss='binary_crossentropy',
                 optimizer = 'adam',
                 metrics = 'accuracy')

modelo_rn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 20)                920       
                                                                 
 dense_5 (Dense)             (None, 16)                336       
                                                                 
 dense_6 (Dense)             (None, 1)                 17        
                                                                 
Total params: 1,273
Trainable params: 1,273
Non-trainable params: 0
_________________________________________________________________


In [19]:
rna_es = tf.keras.callbacks.EarlyStopping(
    monitor="val_auc",
    min_delta=0.01,
    patience = 20,
    verbose=2,
    mode='max',
    restore_best_weights=True)

historico = modelo_rn.fit(x=x_train, 
                      y=y_train,
                      epochs=100,
                      batch_size=32,
                      validation_split=0.1,
                      verbose=1,
                      callbacks=[rna_es])

Epoch 1/100
149/149 [==============================] - 1s 2ms/step - loss: 0.5151 - accuracy: 0.7486 - val_loss: 0.4520 - val_accuracy: 0.7637
Epoch 2/100
149/149 [==============================] - 0s 1ms/step - loss: 0.4389 - accuracy: 0.7921 - val_loss: 0.4345 - val_accuracy: 0.7940
Epoch 3/100
149/149 [==============================] - 0s 1ms/step - loss: 0.4280 - accuracy: 0.7968 - val_loss: 0.4275 - val_accuracy: 0.8034
Epoch 4/100
149/149 [==============================] - 0s 1ms/step - loss: 0.4237 - accuracy: 0.7972 - val_loss: 0.4298 - val_accuracy: 0.8129
Epoch 5/100
149/149 [==============================] - 0s 1ms/step - loss: 0.4213 - accuracy: 0.7997 - val_loss: 0.4342 - val_accuracy: 0.7826
Epoch 6/100
149/149 [==============================] - 0s 1ms/step - loss: 0.4190 - accuracy: 0.7993 - val_loss: 0.4195 - val_accuracy: 0.8110
Epoch 7/100
149/149 [==============================] - 0s 1ms/step - loss: 0.4179 - accuracy: 0.8005 - val_loss: 0.4262 - val_accuracy: 0.8091

Epoch 23/100
149/149 [==============================] - 0s 1ms/step - loss: 0.4057 - accuracy: 0.8096 - val_loss: 0.4131 - val_accuracy: 0.8034
Epoch 24/100
149/149 [==============================] - 0s 1ms/step - loss: 0.4064 - accuracy: 0.8088 - val_loss: 0.4144 - val_accuracy: 0.8091
Epoch 25/100
149/149 [==============================] - 0s 1ms/step - loss: 0.4030 - accuracy: 0.8083 - val_loss: 0.4123 - val_accuracy: 0.8091
Epoch 26/100
149/149 [==============================] - 0s 1ms/step - loss: 0.4026 - accuracy: 0.8077 - val_loss: 0.4141 - val_accuracy: 0.8091
Epoch 27/100
149/149 [==============================] - 0s 1ms/step - loss: 0.4031 - accuracy: 0.8096 - val_loss: 0.4093 - val_accuracy: 0.8072
Epoch 28/100
149/149 [==============================] - 0s 1ms/step - loss: 0.4026 - accuracy: 0.8092 - val_loss: 0.4229 - val_accuracy: 0.8091
Epoch 29/100
149/149 [==============================] - 0s 1ms/step - loss: 0.4023 - accuracy: 0.8096 - val_loss: 0.4121 - val_accuracy:

Epoch 45/100
149/149 [==============================] - 0s 1ms/step - loss: 0.3921 - accuracy: 0.8136 - val_loss: 0.4197 - val_accuracy: 0.8053
Epoch 46/100
149/149 [==============================] - 0s 1ms/step - loss: 0.3916 - accuracy: 0.8138 - val_loss: 0.4081 - val_accuracy: 0.8072
Epoch 47/100
149/149 [==============================] - 0s 1ms/step - loss: 0.3904 - accuracy: 0.8165 - val_loss: 0.4085 - val_accuracy: 0.7958
Epoch 48/100
149/149 [==============================] - 0s 1ms/step - loss: 0.3898 - accuracy: 0.8132 - val_loss: 0.4158 - val_accuracy: 0.8110
Epoch 49/100
149/149 [==============================] - 0s 1ms/step - loss: 0.3891 - accuracy: 0.8140 - val_loss: 0.4200 - val_accuracy: 0.8147
Epoch 50/100
149/149 [==============================] - 0s 1ms/step - loss: 0.3897 - accuracy: 0.8151 - val_loss: 0.4213 - val_accuracy: 0.8072
Epoch 51/100
149/149 [==============================] - 0s 1ms/step - loss: 0.3891 - accuracy: 0.8182 - val_loss: 0.4172 - val_accuracy:

Epoch 67/100
149/149 [==============================] - 0s 1ms/step - loss: 0.3792 - accuracy: 0.8210 - val_loss: 0.4185 - val_accuracy: 0.8147
Epoch 68/100
149/149 [==============================] - 0s 1ms/step - loss: 0.3801 - accuracy: 0.8178 - val_loss: 0.4137 - val_accuracy: 0.8185
Epoch 69/100
149/149 [==============================] - 0s 1ms/step - loss: 0.3795 - accuracy: 0.8233 - val_loss: 0.4206 - val_accuracy: 0.8147
Epoch 70/100
149/149 [==============================] - 0s 1ms/step - loss: 0.3792 - accuracy: 0.8205 - val_loss: 0.4219 - val_accuracy: 0.8147
Epoch 71/100
149/149 [==============================] - 0s 1ms/step - loss: 0.3769 - accuracy: 0.8216 - val_loss: 0.4268 - val_accuracy: 0.8129
Epoch 72/100
149/149 [==============================] - 0s 1ms/step - loss: 0.3780 - accuracy: 0.8207 - val_loss: 0.4182 - val_accuracy: 0.8072
Epoch 73/100
149/149 [==============================] - 0s 1ms/step - loss: 0.3767 - accuracy: 0.8220 - val_loss: 0.4127 - val_accuracy:

Epoch 89/100
149/149 [==============================] - 0s 1ms/step - loss: 0.3693 - accuracy: 0.8235 - val_loss: 0.4278 - val_accuracy: 0.8147
Epoch 90/100
149/149 [==============================] - 0s 1ms/step - loss: 0.3681 - accuracy: 0.8260 - val_loss: 0.4318 - val_accuracy: 0.8110
Epoch 91/100
149/149 [==============================] - 0s 1ms/step - loss: 0.3695 - accuracy: 0.8268 - val_loss: 0.4256 - val_accuracy: 0.8110
Epoch 92/100
149/149 [==============================] - 0s 1ms/step - loss: 0.3672 - accuracy: 0.8260 - val_loss: 0.4291 - val_accuracy: 0.7902
Epoch 93/100
149/149 [==============================] - 0s 1ms/step - loss: 0.3679 - accuracy: 0.8231 - val_loss: 0.4292 - val_accuracy: 0.8166
Epoch 94/100
149/149 [==============================] - 0s 1ms/step - loss: 0.3679 - accuracy: 0.8231 - val_loss: 0.4252 - val_accuracy: 0.8053
Epoch 95/100
149/149 [==============================] - 0s 1ms/step - loss: 0.3661 - accuracy: 0.8264 - val_loss: 0.4237 - val_accuracy:

In [24]:
# Avaliando o modelo no conjunto de teste
loss, accuracy = modelo_rn.evaluate(x_test, y_test)
print(f'Acurácia do modelo no conjunto de teste: {accuracy * 100:.2f}%')

56/56 [==============================] - 0s 871us/step - loss: 0.4264 - accuracy: 0.7973
Acurácia do modelo no conjunto de teste: 79.73%


### Melhorando os hiperparametros

In [25]:
def treina_modelo_rn(x_treino, y_treino, x_teste, y_teste, ncc1, ncc2):
    
    modelo_rn = keras.Sequential()

    # Camada Oculta 1
    modelo_rn.add(layers.Dense(nnc1,
                             activation='tanh',
                              input_dim=x_train.shape[1]))

    # Camada Oculta 2
    modelo_rn.add(layers.Dense(nnc2,
                             activation = 'relu',
                             input_dim = x_train.shape[1]))

    # Camada Saida
    modelo_rn.add(layers.Dense(1,
                              activation = 'sigmoid'))


    #
    modelo_rn.compile(loss='binary_crossentropy',
                     optimizer = 'adam',
                     metrics = 'accuracy')

    
        # Treinamento do Modelo RNA
    rna_es = tf.keras.callbacks.EarlyStopping(
    monitor="val_auc",
    min_delta=0.01,
    patience = 5,
    verbose=0,
    mode='max',
    restore_best_weights=True)

    historico = model.fit(x=x_train, 
                          y=y_train,
                          epochs=100,
                          batch_size=32,
                          validation_split=0.1,
                          verbose=0,
                          callbacks=[rna_es])

IndentationError: expected an indented block (1339553446.py, line 1)